# Supervised Topic Modeling with Labeled LDA

In this notebook, we train a **Labeled Latent Dirichlet Allocation (Labeled LDA)** model to learn topics associated with predefined `Category` labels from report view descriptions. This allows us to:
- Learn label-specific topics from text
- Predict and interpret categories for new or unlabeled views
- Improve semantic tagging of report views


In [20]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import tomotopy as tp

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

# Cargar la hoja "Views"
df = pd.read_excel("../raw/Reporting_Inventory.xlsx", sheet_name="Views")
df = df[df["Description"].notna() & df["Category"].notna() & df["Report View"].notna()]
df.head(2)

[nltk_data] Downloading package punkt to /Users/cbadenes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cbadenes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ID Data Product,Report Name,Product Owner,PBIX_File,Report View,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Tags,Priority
0,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,CRITERIA,Methodolody and definition of the algorithim of Feeder Market,Informative,Productive,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1
1,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,DESTINATION_OF_FEEDER_MARKETS,View focused on understand the performance by hotel for a specific feeder market o selection of feeder marktes.,Functional,Productive,NaN,"Hotel, month, Feeder Market, Segment, Channel Mix, Room Type","Total Revenue, Room Revenue, RN, Lead Time, Lenght of Stay, AOV, ADR, ADR Net, %Cost",NaN,NaN,NaN,Priority 1


In [41]:
# Preprocesamiento
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-záéíóúñü\s]", "", text)
    tokens = word_tokenize(text)
    return [t for t in tokens if t not in stop_words]

#df["tokens"] = df["Description"].apply(preprocess)
#df["tokens"] = df["tokens"] + df["Report View"].apply(preprocess)

df["tokens"] = df["Report View"].apply(preprocess)
df["label"] = df["Category"].str.lower().str.strip()
df.head(2)

,ID Data Product,Report Name,Product Owner,PBIX_File,Report View,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Tags,Priority,tokens,label
0,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,CRITERIA,Methodolody and definition of the algorithim of Feeder Market,Informative,Productive,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1,[criteria],informative
1,RPPBI0032,Feeder Market - 2024,Jonathan Shields,LifeReport.pbix,DESTINATION_OF_FEEDER_MARKETS,View focused on understand the performance by hotel for a specific feeder market o selection of feeder marktes.,Functional,Productive,NaN,"Hotel, month, Feeder Market, Segment, Channel Mix, Room Type","Total Revenue, Room Revenue, RN, Lead Time, Lenght of Stay, AOV, ADR, ADR Net, %Cost",NaN,NaN,NaN,Priority 1,[destinationoffeedermarkets],functional


In [42]:
df["label"].value_counts()

label
functional      360
index            67
executive        57
informative      42
self-service     13
other            12
master data       7
Name: count, dtype: int64

## Train a labeled LDA

* tw=tp.TermWeight.ONE
This sets the term weighting scheme to no weighting (i.e., each word has equal importance).
Alternative values include:   
    * tp.TermWeight.PMI – Pointwise Mutual Information
    * tp.TermWeight.IDF – Inverse Document Frequency
    * tp.TermWeight.TFIDF – Term Frequency–Inverse Document Frequency

* min_cf=3
Minimum collection frequency: a token must appear in at least 3 documents to be included in the vocabulary.
This filters out extremely rare words to reduce noise.

* rm_top=5
Removes the top 5 most frequent tokens from the vocabulary.
These are typically very common terms (e.g. "data", "report") that add little semantic value for topic modeling.

In [45]:
import tomotopy as tp

# Obtener etiquetas únicas de forma segura
unique_labels = sorted(set(df["label"]))

# Crear modelo Labeled LDA
model = tp.PLDAModel(tw=tp.TermWeight.ONE, min_cf=3, rm_top=5, alpha=0.1, eta=0.01)

# Añadir documentos
for tokens, label in zip(df["tokens"], df["label"]):
    model.add_doc(tokens, labels=[label])

# Entrenar progresivamente
model.train(0)
for i in range(100, 1000, 100):
    model.train(100)
    print(f"Log-likelihood after {i} iterations: {model.ll_per_word:.4f}")


Log-likelihood after 100 iterations: -5.3487
Log-likelihood after 200 iterations: -5.3486
Log-likelihood after 300 iterations: -5.3484
Log-likelihood after 400 iterations: -5.3481
Log-likelihood after 500 iterations: -5.3485
Log-likelihood after 600 iterations: -5.3485
Log-likelihood after 700 iterations: -5.3483
Log-likelihood after 800 iterations: -5.3482
Log-likelihood after 900 iterations: -5.3486


## Evaluate the Model

In [46]:
# Ver las palabras más relevantes por categoría
for i, label in enumerate(unique_labels):
    print(f"Top words for label '{label}':")
    print(model.get_topic_words(i, top_n=10))
    print()

Top words for label 'executive':
[('glossary', 0.48243799805641174), ('information', 0.2069559246301651), ('criteria', 0.13808540999889374), ('compset', 0.06921487301588058), ('info', 0.034779611974954605), ('channel', 0.034779611974954605), ('date', 0.0003443525929469615), ('home', 0.0003443525929469615), ('dashboard', 0.0003443525929469615), ('vs', 0.0003443525929469615)]

Top words for label 'functional':
[('performance', 0.03713095933198929), ('dashboard', 0.026822529733181), ('hotel', 0.024760844185948372), ('process', 0.022699158638715744), ('vs', 0.022699158638715744), ('date', 0.022699158638715744), ('fc', 0.020637473091483116), ('management', 0.020637473091483116), ('pet', 0.01857578754425049), ('business', 0.01857578754425049)]

Top words for label 'index':
[('executive', 0.4481392204761505), ('overview', 0.08631978183984756), ('redacted', 0.08631978183984756), ('euam', 0.08631978183984756), ('hotel', 0.06909028440713882), ('overall', 0.034631289541721344), ('ecommerce', 0.03

### Make Inferences

In [48]:
#tokens = preprocess("The reports sent by STR every 3 months with forecast data from some markets of %OCC, ADR and RevPar, are consolidated on this tab.")
tokens = preprocess("glossary of information by criteria kn a compset")
#doc = model.make_doc(tokens)
doc = model.make_doc(tokens, labels=unique_labels) 
topic_dist, _ = model.infer(doc)
    
# Get most probable label
best_label = max(zip(unique_labels, topic_dist), key=lambda x: x[1])[0]
print("Topic:",best_label)

Topic: executive


## Use the model

In [14]:
# Aplicar a vistas sin categoría
unlabeled_df = pd.read_excel("../raw/Reporting_Inventory.xlsx", sheet_name="Views")
unlabeled_df = unlabeled_df[unlabeled_df["Category"].isna() & unlabeled_df["Description"].notna()]
unlabeled_df["tokens"] = unlabeled_df["Description"].apply(preprocess)

# Predecir la categoría más probable
predictions = []
for tokens in unlabeled_df["tokens"]:
    doc = model.make_doc(tokens)
    topic_dist, _ = model.infer(doc)

    # Buscar la mejor etiqueta usando su probabilidad
    best_label = max(zip(unique_labels, topic_dist), key=lambda x: x[1])[0]
    predictions.append(best_label)

unlabeled_df["predicted_category"] = predictions
unlabeled_df.head(10)


,ID Data Product,Report Name,Product Owner,PBIX_File,Report View,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Tags,Priority,tokens,predicted_category
182,RPPBI0034,Corporate Market Share - 2024,Raven Jordan,CharacterReport.pbix,STR Forecast Dashboard 2024,"The reports sent by STR every 3 months with forecast data from some markets of %OCC, ADR and RevPar, are consolidated on this tab.",NaN,Productive,NaN,Cities available,"Occupancy, ADR, RevPar",%Chg last 2 forecast,"Forecast Month, Flag STR is Yes, Hotel_Name is not Hotel Puebla Finsa or Hotel Curitiba The Five or Hotel Lisboa Campo Grande","STR Forecast, Corporate Market Share, 2024",Priority 1,"[reports, sent, str, every, months, forecast, data, markets, occ, adr, revpar, consolidated, tab]",functional
183,RPPBI0034,Corporate Market Share - 2024,Raven Jordan,CharacterReport.pbix,STR Forecast Dashboard 2025,"The reports sent by STR every 3 months with forecast data from some markets of %OCC, ADR and RevPar, are consolidated on this tab.",NaN,Productive,NaN,Cities available,"Occupancy, ADR, RevPar",%Chg last 2 forecast,"Forecast Month, Flag STR is Yes, Hotel_Name is not Hotel Puebla Finsa or Hotel Curitiba The Five or Hotel Lisboa Campo Grande","STR Forecast, Corporate Market Share, 2024",Priority 1,"[reports, sent, str, every, months, forecast, data, markets, occ, adr, revpar, consolidated, tab]",functional
259,RPPBI0150,Corporate Market Share - 2025,Matthew Callahan,SameReport.pbix,STR Forecast Dashboard 2025,"The reports sent by STR every 3 months with forecast data from some markets of %OCC, ADR and RevPar, are consolidated on this tab.",NaN,Productive,NaN,Cities available,"Occupancy, ADR, RevPar",%Chg last 2 forecast,"Forecast Month, Flag STR is Yes, Hotel_Name is not Hotel Puebla Finsa or Hotel Curitiba The Five or Hotel Lisboa Campo Grande","STR Forecast, Corporate Market Share",Priority 1,"[reports, sent, str, every, months, forecast, data, markets, occ, adr, revpar, consolidated, tab]",functional
320,RPPBI0173,Daily Revenue Report 2025,Tasha Hall,AboutReport.pbix,Pick Up Channel Detail,DELETED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1,[deleted],functional
358,RPPBI0062,Price Competitiveness,Nicole Carter,AboutReport.pbix,Booking Criteria,"This view is exclusively for Booking.com,given that they have their offensive criteria. They stablish that a searchis considered offensive when the price difference is greater then 3% and the ranking position is less than 4",NaN,Productive,NaN,"BU, Country, City, Hotel, Brand, META, OTA",NaN,NaN,NaN,NaN,Priority 1,"[view, exclusively, bookingcomgiven, offensive, criteria, stablish, searchis, considered, offensive, price, difference, greater, ranking, position, less]",functional
362,RPPBI0062,Price Competitiveness,Nicole Carter,AboutReport.pbix,Page 1,internal,NaN,Internal,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1,[internal],functional


In [22]:
# Predecir la categoría más probable
predictions = []
for tokens in unlabeled_df["tokens"]:
    doc = model.make_doc(tokens)
    topic_dist, _ = model.infer(doc)

    # Buscar la mejor etiqueta usando su probabilidad
    best_label = max(zip(unique_labels, topic_dist), key=lambda x: x[1])[0]
    predictions.append(best_label)

unlabeled_df["predicted_category"] = predictions
unlabeled_df.head(5)

,ID Data Product,Report Name,Product Owner,PBIX_File,Report View,Description,Category,Status,Rename,Dimensions,KPIs,Other Terms,Filters,Tags,Priority,tokens,predicted_category
182,RPPBI0034,Corporate Market Share - 2024,Raven Jordan,CharacterReport.pbix,STR Forecast Dashboard 2024,The reports sent by STR every 3 months with fo...,NaN,Productive,NaN,Cities available,"Occupancy, ADR, RevPar",%Chg last 2 forecast,"Forecast Month, Flag STR is Yes, Hotel_Name is...","STR Forecast, Corporate Market Share, 2024",Priority 1,"[reports, sent, str, every, months, forecast, ...",functional
183,RPPBI0034,Corporate Market Share - 2024,Raven Jordan,CharacterReport.pbix,STR Forecast Dashboard 2025,The reports sent by STR every 3 months with fo...,NaN,Productive,NaN,Cities available,"Occupancy, ADR, RevPar",%Chg last 2 forecast,"Forecast Month, Flag STR is Yes, Hotel_Name is...","STR Forecast, Corporate Market Share, 2024",Priority 1,"[reports, sent, str, every, months, forecast, ...",functional
259,RPPBI0150,Corporate Market Share - 2025,Matthew Callahan,SameReport.pbix,STR Forecast Dashboard 2025,The reports sent by STR every 3 months with fo...,NaN,Productive,NaN,Cities available,"Occupancy, ADR, RevPar",%Chg last 2 forecast,"Forecast Month, Flag STR is Yes, Hotel_Name is...","STR Forecast, Corporate Market Share",Priority 1,"[reports, sent, str, every, months, forecast, ...",functional
320,RPPBI0173,Daily Revenue Report 2025,Tasha Hall,AboutReport.pbix,Pick Up Channel Detail,DELETED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Priority 1,[deleted],functional
358,RPPBI0062,Price Competitiveness,Nicole Carter,AboutReport.pbix,Booking Criteria,"This view is exclusively for Booking.com,given...",NaN,Productive,NaN,"BU, Country, City, Hotel, Brand, META, OTA",NaN,NaN,NaN,NaN,Priority 1,"[view, exclusively, bookingcomgiven, offensive...",functional
